In [ ]:
from torch.nn import LSTM
import pandas as pd
import numpy as np
from os import getcwd
from os.path import join
import sys

sys.path.insert(0, join(getcwd(), "../module_code"))
from utils import get_preprocessed_file_name, load_cli_args, init_cli_args

sys.argv = [sys.argv[0]]

In [ ]:
load_cli_args("../options.yml")
args = init_cli_args()
preprocessed_df_fname = get_preprocessed_file_name(
    args.time_before_start_date,
    args.time_interval,
    args.preprocessed_df_file,
    serialization=args.serialization,
)
preprocessed_df_path = join(args.raw_data_dir, preprocessed_df_fname)

deserialize_fn = getattr(pd, f"read_{args.serialization}")

In [ ]:
df = deserialize_fn(preprocessed_df_path)

In [ ]:
df = df.groupby(["IP_PATIENT_ID", "DATE"]).last()
df

In [ ]:
df.index.get_level_values("DATE").isnull().sum()

In [ ]:
# keep only numeric columns
df = df.select_dtypes(["number"])

In [ ]:
grouped_sample_dfs = df.groupby(level="IP_PATIENT_ID")
n_samples = grouped_sample_dfs.ngroups  # number of patients
seq_lengths = grouped_sample_dfs.size().values  # seq len per patient
n_features = df.shape[1]  # number of features per pt per seq entry
pad_value = -1
longest_seq = max(seq_lengths)

In [ ]:
# Create empty matrix with padding tokens
padded = np.ones((n_samples, longest_seq, n_features)) * pad_value
# copy over the actual sequences
start = 0
for i, seq_len in enumerate(seq_lengths):
    sequence = df.iloc[start : start + seq_len].values
    if sequence.shape[0] != seq_len: print("bad")
    padded[i, 0:seq_len] = sequence
    # start += seq_len
padded


In [ ]:
from torch import Tensor
Tensor(padded)

In [ ]:
target_index = df.columns.get_loc("recommend_crrt")
padded[:,:,target_index]
np.delete(padded, target_index, axis=-1)